 <img src="Images/fc.pos.jpg" class="bg-primary" align = 'left' width=200/>


# Academia de Física 2025
Neste notebook mostramos um exemplo de análise de dados usando as facilidades do `Pandas`.
A experiência foi realizada num projeto com estudantes do secundário, [projeto Faraday](http://www.fc.up.pt/faraday).

### Descrição da experiência

O esquema da experiência está representado na figura:



<img src="Images/queda.png" width=400 align=center />

Um pequeno cilindro desce em queda livre e passa numa *photogate* (célula fotoelétrica) ligada a um cronómetro; este regista o tempo, $\Delta t$,  durante o qual o cilindro interrompe o feixe.
A velocidade do cilindro na photogate é dada por 

$$ v=\frac{d_c}{\Delta t}$$

em que $d_c=5.07$ cm é a altura do cilindro. De acordo com a lei de conservação de energia

$$ \frac{1}{2}mv^2=mgh$$ 
    
em que $h$ é a distância de queda. Ou 

$$
v^2(h)=2gh
$$

É esta relação que foi estudada, variando $h$. Vamos carregar os dados de um ficheiro excel e proceder à análise. Veremos que a resistância do ar afeta os resultados da experiência.

In [ ]:
%matplotlib inline                                     
import numpy as np                    # modulo numérico , essencial para trabalho cientifico; chamado
import matplotlib.pyplot as plt
import pandas as pd

Os dados estão no ficheiro `queda_cilindro.xls`  na pasta `Dados` 
Estão em 7 colunas. A primeira é a altura de queda do cilindro. As seis seguintes  são as medições dos tempos de passagem do cilindro numa photogate para cada altura. A altura do cilindro é de 5.07 cm. 

In [ ]:
alt_cil=.0507
df0=pd.read_excel("Dados/queda_cilindro.xls")
df0

Acontece que a altura foi medida entre o fundo do cilindro na posição inicial e a photogate. A altura efetiva de queda, para a velocidade medida pela photogate,  é esta altura mais metade do comprimento do cilindro. Não precisamos de ir ao `Excel`  fazer esta alteração. 

In [ ]:
df0["altura medida"]=df0["altura medida"]+alt_cil/2
df0

Agora juntamos um coluna com a média de tempos para cada altura e outra com as velocidades. Recorda que para aceder a uma linha de um data frame usamos `dataframe.loc()`. 

In [ ]:
# tempos medios e velocidades
avg_times=np.zeros(7,float)   # lista para os tempos
for r in range(7):
    avg_times[r]=np.average(df0.loc[r][1:7])        # elemento 0 da linha é altura
#    
df0["t_medio"]=avg_times                           # junta um coluna de nome t_medio
df0["velocs"]=alt_cil/avg_times                    # junta uma coluna de nome velocs
df0

Como vês, na linha de indice 5 o cálculo falhou porque falta um dado: só foram registados 5 tempos e não 6. 

<div class="alert alert-block alert-warning">
    <b>Exercício 1</b>
    <p> Vais ter de modificar o código acima; quando o índice $r=5$ a média só pode incluir as colunas 1 a 5. Vê o exemplo da instrução `if... else` e adapta-a para conseguires o tempo médio também na linha 5.</p>
    
</div>

In [ ]:
for r in range(5):
    if r==3:
        print('XXX')
    else:
        print(r)


In [ ]:
# faz aqui o teu exercício

<div class="alert alert-block alert-warning">
    <b>Exercício 2</b>
    <p> Em vez de calcular o tempo médio podíamos calcular para cada tempo a velocidade $$ v_i =\frac{d_c}{t_i}$$ e fazer a média destes valores. Junta mais uma coluna de velocidades ao <i>DataFrame</i> calculadas deste modo. </p>
    
</div>

In [ ]:
# faz aqui o teu exercício

Deverás reparar que as velocidades calculadas por estes dois métodos não são iguais. Uma questão interessante é saber qual destes métodos é correto. Não vamos discutir esta questão aqui, mas deixar apenas a indicação que,  quando a incerteza relativa dos valores calculados é pequena, a diferença entre os dois valores é bastante menor que a incerteza de cada um deles. Daqui para a frente vamos usar a primeira coluna de velocidades. Mas poderás facilmente refazer a análise usando a segunda. 

Antes de prosseguir vamos guardar um ficheiro excel modificado. Verifica que foi criado no teu disco um novo ficheiro.

In [ ]:
df0.to_excel('Dados/queda_dados.xlsx',sheet_name='queda')

Antes de representar os dados juntamos mais uma informação, que não requer uma medição: para uma altura de queda nula a velocidade é nula também. Para isso usamos a função `concat` (de *concatenação*), que nos permite juntar duas `Panda Series`. 

In [ ]:
alturas=pd.concat([pd.Series([0]), df0["altura medida"] ])
v2=pd.concat([pd.Series([0]), df0["velocs"]**2 ])
v2

Um gráfico dos dados:

In [ ]:
alturas=pd.concat([pd.Series([0]), df0["altura medida"] ])
v2=pd.concat([pd.Series([0]), df0["velocs"]**2 ])
plt.plot(alturas, v2, 'b+',ms=6)
plt.xlabel(r'$h / m$', fontsize=14)
plt.ylabel(r'$v^2 / m^2 s^{-2}$',fontsize=14)
plt.grid(True)

Sem resistência do ar, temos
$$
v^2(h) =2gh
$$

e um ajuste linear tem um declive de $2g$. 

Recorda a função `polyfit`  no notebook `Gráficos_estudante.ipynpb`. 

In [ ]:
a,b=np.polyfit(alturas,v2,1)
a/2

In [ ]:
def fit_lin(x):
    a,b=np.polyfit(alturas,v2,1)
    return a*x+b

In [ ]:
plt.plot(alturas, v2, 'b+', label='exp',ms=6)
plt.plot(alturas, fit_lin(alturas),'r-',label='fit',lw=1)
plt.xlabel(r'$h / m$', fontsize=14)
plt.ylabel(r'$v^2 / m^2 s^{-2}$',fontsize=14)
plt.legend()
plt.grid(True)

Se olhares cuidadosamente , verás que os dados mostram curvatura e não ajusta bem ao modelo linear. 

Levando em conta uma força de resistância do ar, 

$$f=-\gamma v^2, $$

obtém-se uma lei de movimento diferente

$$ v^2 (h)=\frac{mg}{\gamma}\left(1-e^{-2(\gamma/m)h}\right)$$ 

que é uma função de dois parâmetros $g$ e $\beta=\gamma/m$

$$ v^2 (h)=\frac{g}{\beta}\left(1-e^{-2\beta h}\right)$$ 

Para ajustar esta função temos que usar um ajuste não linear porque a função não é linear no parâmetro $\beta$. A função `curve_fit` do módulo `scipy.optimize` permite encontrar os valores dos parâmetros $g,\beta$ que minimizam a soma dos quadrados dos desvios dos dados ao modelo.
$$
S(g,\beta)=\sum_i \left(y_i-f(x_i,g,\beta)\right)^2
$$
Esta função tem 4 argumentos:

 - o modelo a ajustar, $f$;
 - os valores das abcissas, as alturas $x_i$;
 - os valores das ordenadas, as velocidades ao quadrado, $y_i$;
 - como é um ajuste não-linear tem de ter uma proposta para os valores dos parâmetros para iniciar a otimização dos mesmos.
 
Devolve os parâmetros e uma matriz, cujos elementos diagonais são estimativas das variâncias dos dois parâmetros. Vejamos como funciona neste caso. 


In [ ]:
# carregar a função
from scipy.optimize import curve_fit

In [ ]:
# o modelo
def f2(h,g,beta):
    return g/beta*(1-np.exp(-2*beta*h))

In [ ]:
# o ajuste
# valores propostos g=10,beta=1
parms,cov=curve_fit(f2,alturas,v2,p0=np.array([10,1]))
parms


In [ ]:
parms,cov=curve_fit(f2,alturas,v2,p0=np.array([10,.1]))
parms

In [ ]:
cov

Podemos estimar as incertezas nos dois parâmetros:

In [ ]:
std=np.sqrt(np.diag(cov))
std

<div class="alert alert-block alert-warning">
    <b>Exercício 3</b>
    <p> Produz um gráfico dos dados sobreposto à função de ajuste, com os parâmetros obtidos pelo método dos mínimos quadrados. O ajuste parece-te melhor que o linear?
</div>

In [ ]:
# faz aqui o teu exercício

<div class="alert alert-block alert-warning">
    <b>Exercício 4</b>
    <p> Refaz a análise anterior usando as velocidades calculadas com o segundo método (coluna <b>media_v</b>). Compara as diferenças dos valores obtidos para os parâmetros com a respetiva incerteza.  
</div>

In [ ]:
# faz aqui o teu exercício

### Referências

 1. Pandas tutorial,  W3Schools [Pandas tutorial](http://www.w3schools.com/python/pandas/)
 